# Clustering Crypto

In [163]:
# Initial imports
import pandas as pd
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px
import numpy as np

### Deliverable 1: Preprocessing the Data for PCA

In [2]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_data_df = pd.read_csv(file_path, index_col=0)
crypto_data_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [3]:
# Keep all the cryptocurrencies that are being traded.
df = crypto_data_df[crypto_data_df.IsTrading.eq(True)]
df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [4]:
# Remove the "IsTrading" column. 
df = df.drop(columns= "IsTrading")
df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [5]:
# Remove rows that have at least 1 null value.
df = df.dropna()
df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [6]:
# Keep the rows where coins are mined.
df = df[df['TotalCoinsMined'] > 0] 
df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [7]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = df[['CoinName']]
names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [8]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df = df.drop(columns='CoinName')
df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [9]:
# Use get_dummies() to create variables for text features.
dummies_df = pd.get_dummies(df, columns=['Algorithm', 'ProofType'])
dummies_df

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Standardize the data with StandardScaler().
df_scaled = StandardScaler().fit_transform(dummies_df)
print(df_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [11]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
data_pca = pca.fit_transform(df_scaled)
data_pca

array([[-0.33594587,  1.03863778, -0.51232837],
       [-0.31929194,  1.03894328, -0.51247069],
       [ 2.31000119,  1.72521864, -0.53957689],
       ...,
       [ 0.32260973, -2.23386792,  0.38372322],
       [-0.12997373, -2.25815584,  0.38943772],
       [-0.29025681,  0.75570098, -0.22204565]])

In [12]:
# Create a DataFrame with the three principal components.
df_pca = pd.DataFrame(
data=data_pca, index=dummies_df.index, columns=["PC 1", "PC 2", "PC 3"])
df_pca.head(10)

,PC 1,PC 2,PC 3
42,-0.335946,1.038638,-0.512328
404,-0.319292,1.038943,-0.512471
1337,2.310001,1.725219,-0.539577
BTC,-0.148099,-1.317958,0.162330
ETH,-0.150670,-2.086743,0.330716
LTC,-0.169053,-1.136015,-0.020117
DASH,-0.397031,1.353989,-0.471284
XMR,-0.157566,-2.126872,0.322208
ETC,-0.149113,-2.086824,0.330718
ZEC,-0.129973,-2.258156,0.389438


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [13]:
import hvplot.pandas
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_pca)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve") 

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [14]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_pca)

# Predict clusters
predictions = model.predict(df_pca)

predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [149]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
concat_df = pd.concat([df, df_pca], axis=1, join="inner")
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([concat_df,names_df], axis=1, join="inner")
clustered_df
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# # Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335946,1.038638,-0.512328,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319292,1.038943,-0.512471,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.310001,1.725219,-0.539577,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148099,-1.317958,0.162330,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.150670,-2.086743,0.330716,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.169053,-1.136015,-0.020117,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.397031,1.353989,-0.471284,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.157566,-2.126872,0.322208,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.149113,-2.086824,0.330718,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.129973,-2.258156,0.389438,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [150]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", hover_name="CoinName", hover_data=['Algorithm'], symbol="Class",width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [151]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName','Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [152]:
# Print the total number of tradable cryptocurrencies.
tradable_number = len(clustered_df)
f'There are {tradable_number} tradable cryptocurrencies'

'There are 532 tradable cryptocurrencies'

In [172]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
data_to_scale = pd.DataFrame(clustered_df.drop(columns=['Algorithm', 'ProofType', 
                                                        'PC 1', 'PC 2', 'PC 3', 'CoinName', 'Class']))
scaled_data = MinMaxScaler().fit_transform(data_to_scale)
scaled_data

array([[0.00000000e+00, 4.20000000e-11],
       [1.06585544e-03, 5.32000000e-04],
       [2.95755135e-02, 3.14159265e-01],
       ...,
       [9.90135079e-04, 1.40022261e-03],
       [7.37028150e-06, 2.10000000e-05],
       [1.29582282e-07, 1.00000000e-06]])

In [208]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
data = pd.DataFrame({'TotalCoinSupply1': scaled_data[:, 1], 'TotalCoinsMined2': scaled_data[:, 0]})

clustered_df['TotalCoinSupply1'] = scaled_data[:, 1]
clustered_df['TotalCoinsMined2'] = scaled_data[:, 0]

clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class,TotalCoinSupply1,TotalCoinsMined2
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335946,1.038638,-0.512328,42 Coin,0,4.200000e-11,0.000000e+00
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319292,1.038943,-0.512471,404Coin,0,5.320000e-04,1.065855e-03
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.310001,1.725219,-0.539577,EliteCoin,0,3.141593e-01,2.957551e-02
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148099,-1.317958,0.162330,Bitcoin,3,2.100000e-05,1.810842e-05
ETH,Ethash,PoW,1.076842e+08,0,-0.150670,-2.086743,0.330716,Ethereum,3,0.000000e+00,1.087731e-04
...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.475675,0.782640,0.037223,ZEPHYR,0,2.000000e-03,2.020225e-03
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.333992,1.038546,-0.512328,Gapcoin,0,2.500000e-04,1.508199e-05
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.322610,-2.233868,0.383723,Beldex,3,1.400223e-03,9.901351e-04
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.129974,-2.258156,0.389438,Horizen,3,2.100000e-05,7.370282e-06


In [209]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_df.hvplot.scatter(
    x="TotalCoinsMined2",
    y="TotalCoinSupply1",
    hover_cols=["CoinName"], 
    by="Class")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined2]   (TotalCoinSupply1,CoinName)